In [1]:
using Distributed

In [2]:
addprocs(4)

4-element Vector{Int64}:
 2
 3
 4
 5

In [3]:
x = workers()

4-element Vector{Int64}:
 2
 3
 4
 5

In [4]:
x

4-element Vector{Int64}:
 2
 3
 4
 5

In [5]:
@everywhere using DistributedArrays

In [6]:
D = DArray([@spawnat p [(woker = p, flag = Ref(false))] for p in workers()])

4-element DArray{@NamedTuple{woker::Int64, flag::Base.RefValue{Bool}}, 1, Vector{@NamedTuple{woker::Int64, flag::Base.RefValue{Bool}}}}:
 (woker = 2, flag = Base.RefValue{Bool}(false))
 (woker = 3, flag = Base.RefValue{Bool}(false))
 (woker = 4, flag = Base.RefValue{Bool}(false))
 (woker = 5, flag = Base.RefValue{Bool}(false))

In [7]:
[@fetchfrom p localindices(D) for p in workers()]

4-element Vector{Tuple{UnitRange{Int64}}}:
 (1:1,)
 (2:2,)
 (3:3,)
 (4:4,)

In [8]:
[@fetchfrom p localpart(D) for p in workers()]

4-element Vector{Vector{@NamedTuple{woker::Int64, flag::Base.RefValue{Bool}}}}:
 [(woker = 2, flag = Base.RefValue{Bool}(false))]
 [(woker = 3, flag = Base.RefValue{Bool}(false))]
 [(woker = 4, flag = Base.RefValue{Bool}(false))]
 [(woker = 5, flag = Base.RefValue{Bool}(false))]

In [9]:
for p in workers()
    @fetchfrom p only(localpart(D)).flag[]=true
end

In [10]:
D

4-element DArray{@NamedTuple{woker::Int64, flag::Base.RefValue{Bool}}, 1, Vector{@NamedTuple{woker::Int64, flag::Base.RefValue{Bool}}}}:
 (woker = 2, flag = Base.RefValue{Bool}(true))
 (woker = 3, flag = Base.RefValue{Bool}(true))
 (woker = 4, flag = Base.RefValue{Bool}(true))
 (woker = 5, flag = Base.RefValue{Bool}(true))

In [11]:
D[2].flag[]

true

In [12]:
t = @async begin
    sleep(10)
    1
end

Task (runnable) @0x00007faeceae8e20

In [13]:
t

Task (runnable) @0x00007faeceae8e20

In [14]:
istaskdone(t)

false

In [15]:
while true
    println("Waiting")
    if timedwait(()->istaskdone(t), 100) == :ok
        println("Done waiting: $(fetch(t))")
        break
    end
end

Waiting
Done waiting: 1


In [16]:
fetch(t)

1

In [17]:
using Distributed

In [18]:
addprocs(4)

4-element Vector{Int64}:
 6
 7
 8
 9

In [20]:
@everywhere function test1()
    w=workers()
    println("workers=$(w)")
end
@everywhere test2() = println("myid=$(myid())")
@everywhere function test3()
    println("myid=$(myid()), running test2")
    remote_do(test2, 2)
end

@everywhere function test4(channel)
    println("taking from channel")
    x = take!(channel)
    println("x=$(x)")
end

In [21]:
test1()

workers=[2, 3, 4, 5, 6, 7, 8, 9]


In [22]:
test2()

myid=1


In [23]:
test3()

myid=1, running test2
      From worker 2:	myid=2


In [24]:
remote_do(test1, 2)

      From worker 2:	workers=[2, 3, 4, 5, 6, 7, 8, 9]


In [8]:
remote_do(test3, 3)

      From worker 3:	myid=3, running test2
      From worker 2:	myid=2


In [9]:
addprocs(2)

2-element Vector{Int64}:
 6
 7

In [11]:
remote_do(test1, 3)

      From worker 3:	workers=[3, 2, 4, 5, 6, 7]


In [12]:
ch = RemoteChannel(()->Channel{Int64}(32))

RemoteChannel{Channel{Int64}}(1, 1, 88)

In [21]:
put!(ch, 1)

RemoteChannel{Channel{Int64}}(1, 1, 88)

      From worker 6:	x=1


In [14]:
workers()

6-element Vector{Int64}:
 2
 3
 4
 5
 6
 7

In [20]:
remote_do(test4, 6, ch)

      From worker 6:	taking from channel


In [17]:
rmprocs(7)

Task (done) @0x00007f10ab57a720

In [18]:
workers()

5-element Vector{Int64}:
 2
 3
 4
 5
 6